In [1]:
import cv2
import imageio
import numpy as np
from scipy.spatial import Delaunay


In [2]:
# Get 468 tie points for human faces using mediapipe

import mediapipe as mp

def find_landmarks_mp(image):
    # Initialize MediaPipe Face Mesh.
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True,
                                      max_num_faces=1,
                                      refine_landmarks=True,
                                      min_detection_confidence=0.5)
    
    # Convert the BGR image to RGB as MediaPipe expects RGB input.
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Process the image to detect face landmarks.
    results = face_mesh.process(rgb_image)
    
    arr = []
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            print(type(face_landmarks))  # Similar to the print statement in the original code.
            height, width, _ = image.shape
            
            # Loop through each detected landmark and convert normalized coordinates to pixel coordinates.
            for landmark in face_landmarks.landmark:
                x = int(landmark.x * width)
                y = int(landmark.y * height)
                arr.append([x, y])
    
    # Add image corners for smoother boundaries.
    arr.append([0, 0])
    arr.append([0, image.shape[0]])
    arr.append([image.shape[1], 0])
    arr.append([image.shape[1], image.shape[0]])
    
    return arr


ModuleNotFoundError: No module named 'mediapipe'

In [ ]:
"""
# Get 68 landmark points using dlib


import dlib

def find_landmarks_dlib(image):
    # Path to the shape predictor file and input image
    PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
    
    # Initialize dlib's face detector (HOG-based) and create the landmark predictor
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(PREDICTOR_PATH)
    
    # Load the image and convert it to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the grayscale image
    faces = detector(gray)
    arr = []
    # Loop over each face detected
    for face in faces:
        # Determine the facial landmarks for the face region
        landmarks = predictor(gray, face)
        print(type(landmarks))
        
        # Loop through each of the 68 facial landmarks and draw them on the image
        for i in range(0, 68):
            x = landmarks.part(i).x
            y = landmarks.part(i).y
            arr.append([x, y])
    # Add the corners for smoother. REMINDER TO TRYR ADDING MIDPOINTS!!!!!!!!!
    arr.append([0,0])
    arr.append([0,image.shape[0]])
    arr.append([image.shape[1], 0])
    arr.append([image.shape[1], image.shape[0]])

    return arr
"""


In [ ]:
def get_point(image, window_title, existing_points):
    """
    1. Shows the image with the previously marked points for reference
    2. Waits for click input of a new point or q to exiit
    """
    # Create a copy of the image and draw the existing points.
    disp_img = image.copy()
    for pt in existing_points:
        cv2.circle(disp_img, pt, 3, (0, 255, 0), -1)
    
    new_point = []

    def on_mouse(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            new_point.append((x, y))

    cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)
    cv2.setMouseCallback(window_title, on_mouse)
    cv2.imshow(window_title, disp_img)
    
    # Wait until a point is clicked or 'q' is pressed.
    while True:
        key = cv2.waitKey(1) & 0xFF
        # If a point has been clicked, break out.
        if new_point:
            break
        # If 'q' is pressed, break out.
        if key == ord('q'):
            break

    cv2.destroyWindow(window_title)
    return new_point[0] if new_point else None

def mpa(src_image_path, dst_image_path):
    """
    1. We get list of landmark points from mediapipe
    2. We extend that list with click input
    """
    src_img = cv2.imread(src_image_path)
    dst_img = cv2.imread(dst_image_path)
    
    if src_img is None or dst_img is None:
        raise ValueError("Could not load one or both images.")

    src_points = find_landmarks_mp(src_img)
    dst_points = find_landmarks_mp(dst_img)
    if len(src_points) != len(dst_points):
        src_points = []
        dst_points = []
    
        # Append the corners by default
        src_points.append([0,0])
        src_points.append([0,src_img.shape[0]])
        src_points.append([src_img.shape[1], 0])
        src_points.append([src_img.shape[1], src_img.shape[0]])
        dst_points.append([0,0])
        dst_points.append([0,dst_img.shape[0]])
        dst_points.append([dst_img.shape[1], 0])
        dst_points.append([dst_img.shape[1], dst_img.shape[0]])
        
    
    print("For each pair, click on the SOURCE image then the corresponding point on the DESTINATION image.")
    
    print("Press 'q' in any window to finish input.")
    
    while True:
        # Get a point from the source image.
        pt_src = get_point(src_img, "Source Image", src_points)
        if pt_src is None:
            print("Finished input for source image.")
            break
        src_points.append(pt_src)
        print(f"Selected source point: {pt_src}")
        
        # Get the corresponding point from the destination image.
        pt_dst = get_point(dst_img, "Destination Image", dst_points)
        if pt_dst is None:
            print("Finished input for destination image.")
            break
        dst_points.append(pt_dst)
        print(f"Selected destination point: {pt_dst}")

 
    
    return src_points, dst_points

def dliba(src_image_path, dst_image_path):
    """
    1. We get list of landmark points from dlib
    2. We extend that list with click input
    """
    src_img = cv2.imread(src_image_path)
    dst_img = cv2.imread(dst_image_path)
    
    if src_img is None or dst_img is None:
        raise ValueError("Could not load one or both images.")

    src_points = find_landmarks_dlib(src_img)
    dst_points = find_landmarks_dlib(dst_img)
    if len(src_points) != len(dst_points):
        src_points = []
        dst_points = []
    
        # Append the corners by default
        src_points.append([0,0])
        src_points.append([0,src_img.shape[0]])
        src_points.append([src_img.shape[1], 0])
        src_points.append([src_img.shape[1], src_img.shape[0]])
        dst_points.append([0,0])
        dst_points.append([0,dst_img.shape[0]])
        dst_points.append([dst_img.shape[1], 0])
        dst_points.append([dst_img.shape[1], dst_img.shape[0]])
        
    
    print("For each pair, click on the SOURCE image then the corresponding point on the DESTINATION image.")
    print("Press 'q' in any window to finish input.")
    
    while True:
        # Get a point from the source image.
        pt_src = get_point(src_img, "Source Image", src_points)
        if pt_src is None:
            print("Finished input for source image.")
            break
        src_points.append(pt_src)
        print(f"Selected source point: {pt_src}")
        
        # Get the corresponding point from the destination image.
        pt_dst = get_point(dst_img, "Destination Image", dst_points)
        if pt_dst is None:
            print("Finished input for destination image.")
            break
        dst_points.append(pt_dst)
        print(f"Selected destination point: {pt_dst}")

 
    
    return src_points, dst_points

def clickity(src_image_path, dst_image_path):
    """
    Takes the click inputs and returns the list
    """
    src_img = cv2.imread(src_image_path)
    dst_img = cv2.imread(dst_image_path)
    
    if src_img is None or dst_img is None:
        raise ValueError("Could not load one or both images.")

    src_points = []
    dst_points = []

    # Append the corners by default
    src_points.append([0,0])
    src_points.append([0,src_img.shape[0]])
    src_points.append([src_img.shape[1], 0])
    src_points.append([src_img.shape[1], src_img.shape[0]])
    dst_points.append([0,0])
    dst_points.append([0,dst_img.shape[0]])
    dst_points.append([dst_img.shape[1], 0])
    dst_points.append([dst_img.shape[1], dst_img.shape[0]])
        
    
    print("For each pair, click on the SOURCE image then the corresponding point on the DESTINATION image.")
    print("Press 'q' in any window to finish input.")
    
    while True:
        # Get a point from the source image.
        pt_src = get_point(src_img, "Source Image", src_points)
        if pt_src is None:
            print("Finished input for source image.")
            break
        src_points.append(pt_src)
        print(f"Selected source point: {pt_src}")
        
        # Get the corresponding point from the destination image.
        pt_dst = get_point(dst_img, "Destination Image", dst_points)
        if pt_dst is None:
            print("Finished input for destination image.")
            break
        dst_points.append(pt_dst)
        print(f"Selected destination point: {pt_dst}")

 
    
    return src_points, dst_points

In [ ]:

def warp_triangle(src_img, dst_img, out_img, tri_src, tri_dst, tri_interp, blend):
    """
    Warp one triangle from the source and destination images into an intermediate shape
    and blend their colors. 
    """
    # Convert triangles to NumPy arrays
    tri_interp = np.array(tri_interp, dtype=np.float32)
    tri_src    = np.array(tri_src, dtype=np.float32)
    tri_dst    = np.array(tri_dst, dtype=np.float32)
    
    # Get bounding box for the intermediate triangle
    x, y, w, h = cv2.boundingRect(tri_interp)
    
    # Create a grid of (x, y) coordinates for the bounding box; add 0.5 for pixel centers
    grid_x, grid_y = np.meshgrid(np.arange(x, x+w), np.arange(y, y+h))
    grid_points = np.vstack((grid_x.ravel(), grid_y.ravel())).T + 0.5  # shape (N, 2)
    
    # Unpack vertices of the intermediate triangle
    A, B, C = tri_interp
    
    # Compute denominator for barycentric coordinates (same for all points)
    denom = ( (B[1]-C[1])*(A[0]-C[0]) + (C[0]-B[0])*(A[1]-C[1]) )
    if np.abs(denom) < 1e-6:
        return  # Avoid division by zero for degenerate triangles.
    
    # Vectorized computation of barycentrics for all grid points
    w1 = ((B[1]-C[1])*(grid_points[:,0]-C[0]) + (C[0]-B[0])*(grid_points[:,1]-C[1])) / denom
    w2 = ((C[1]-A[1])*(grid_points[:,0]-C[0]) + (A[0]-C[0])*(grid_points[:,1]-C[1])) / denom
    w3 = 1 - w1 - w2
    
    # Mask: select only points that are inside the intermediate triangle
    mask = (w1 >= 0) & (w2 >= 0) & (w3 >= 0)
    if np.sum(mask) == 0:
        return  # No valid points.
    
    valid_points = grid_points[mask]      # (M, 2)
    valid_w1 = w1[mask][:, np.newaxis]      # (M, 1)
    valid_w2 = w2[mask][:, np.newaxis]
    valid_w3 = w3[mask][:, np.newaxis]
    
    # Map points to corresponding positions in source and destination triangles
    src_coords = valid_w1 * tri_src[0] + valid_w2 * tri_src[1] + valid_w3 * tri_src[2]
    dst_coords = valid_w1 * tri_dst[0] + valid_w2 * tri_dst[1] + valid_w3 * tri_dst[2]
    
    # Vectorized bilinear interpolation
    def bilinear_interp_vec(img, coords):
        h_img, w_img, channels = img.shape
        x = coords[:, 0]
        y = coords[:, 1]
        x0 = np.floor(x).astype(np.int32)
        x1 = x0 + 1
        y0 = np.floor(y).astype(np.int32)
        y1 = y0 + 1

        # Clip coordinates to image boundaries.
        x0 = np.clip(x0, 0, w_img - 1)
        x1 = np.clip(x1, 0, w_img - 1)
        y0 = np.clip(y0, 0, h_img - 1)
        y1 = np.clip(y1, 0, h_img - 1)

        Ia = img[y0, x0]  # Top-left
        Ib = img[y0, x1]  # Top-right
        Ic = img[y1, x0]  # Bottom-left
        Id = img[y1, x1]  # Bottom-right

        # Compute interpolation weights.
        wa = ((x1 - x) * (y1 - y))[:, np.newaxis]
        wb = ((x - x0) * (y1 - y))[:, np.newaxis]
        wc = ((x1 - x) * (y - y0))[:, np.newaxis]
        wd = ((x - x0) * (y - y0))[:, np.newaxis]

        return wa * Ia + wb * Ib + wc * Ic + wd * Id

    src_colors = bilinear_interp_vec(src_img, src_coords)
    dst_colors = bilinear_interp_vec(dst_img, dst_coords)
    
    # Blend colors from source and destination
    blended_colors = (1 - blend) * src_colors + blend * dst_colors
    
    # Write blended colors back into output image
    # Convert floating coordinates to integer indices (subtract 0.5 to reverse our earlier offset)
    valid_points_int = np.round(valid_points - 0.5).astype(np.int32)
    xs = valid_points_int[:, 0]
    ys = valid_points_int[:, 1]
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! TEST THIS !!!!!!!!!!!!!!!!!!!!!!!!!
    h_img, w_img, _ = out_img.shape
    xs = np.clip(xs, 0, w_img - 1)
    ys = np.clip(ys, 0, h_img - 1)

    out_img[ys, xs] = blended_colors

def morph_images(src_img, dst_img, pts_src, pts_dst, num_frames=30, gif_name='morph.gif'):
    # Convert images to float32 for processing.
    src_img = src_img.astype(np.float32)
    dst_img = dst_img.astype(np.float32)
    pts_src = np.array(pts_src, np.float32)
    pts_dst = np.array(pts_dst, np.float32)
    
    # Compute Delaunay triangulation using source points.
    delaunay = Delaunay(pts_src)
    triangles = delaunay.simplices  # Use these triangle indices for both images.
    
    frames = []
    h, w = src_img.shape[:2]
    
    # Loop over each frame.
    for f in range(num_frames + 1):
        blend = f / num_frames  # 0 means source, 1 means destination.
        morphed_frame = np.zeros_like(src_img)
        
        # Process each triangle.
        for tri_indices in triangles:
            tri_src = [pts_src[i] for i in tri_indices]
            tri_dst = [pts_dst[i] for i in tri_indices]
            # Compute the intermediate triangle by linearly blending vertex positions.
            tri_interp = [
                (1 - blend) * np.array(tri_src[i]) + blend * np.array(tri_dst[i])
                for i in range(3)
            ]
            warp_triangle(src_img, dst_img, morphed_frame, tri_src, tri_dst, tri_interp, blend)
        
        # Clip pixel values and convert to uint8.
        frame_uint8 = np.clip(morphed_frame, 0, 255).astype(np.uint8)
        frames.append(frame_uint8)
        print(f"Frame {f}/{num_frames} complete.")
    
    # Save all frames as an animated GIF.
    imageio.mimsave(gif_name, frames, duration=0.025)
    print(f"Saved GIF as {gif_name}")

In [ ]:
def read_ties_from_file(file_path):
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]

    # The first line should contain T
    T = int(lines[0])

    tie_points_A = []
    tie_points_B = []

    # Next T lines each contain xA, yA, xB, yB
    for i in range(1, T + 1):
        coords = lines[i].split()
        if len(coords) != 4:
            raise ValueError(f"Line {i} doesn't have exactly 4 values: {lines[i]}")
        
        xA = float(coords[0])
        yA = float(coords[1])
        xB = float(coords[2])
        yB = float(coords[3])
        
        tie_points_A.append((xA, yA))
        tie_points_B.append((xB, yB))

    return tie_points_A, tie_points_B


In [ ]:
'''
def read_points_from_file(file_path, delimiter=' '):
    points = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if not line:
                # Skip empty lines
                continue

            # If a delimiter is specified, use it. Otherwise, try to auto-detect space or comma.
            if delimiter is not None:
                coords = line.split(delimiter)
            else:
                # Attempt to split on space first, then fallback to comma
                if ' ' in line:
                    coords = line.split()
                else:
                    coords = line.split(',')

            if len(coords) == 2:
                x, y = float(coords[0]), float(coords[1])
                points.append((x, y))
            else:
                raise ValueError(f"Line format is invalid: '{line}'")

    return points

def read_tie_points(tie1_path='tie1.txt', tie2_path='tie2.txt'):
    """
    Reads tie1.txt and tie2.txt each containing an array of points and returns two lists of (x, y) tuples.
    """
    tie1_points = read_points_from_file(tie1_path)
    tie2_points = read_points_from_file(tie2_path)
    return tie1_points, tie2_points
'''

In [ ]:
def main(src_image_path, dst_image_path, tie_points_acquisition="mp"):
    """
    tie_points_acquisition can be:
    
        1. mp : Use mediapipe to find the landmark poitns
        2. dlib: Use dlib to find the landmark points.
        3. mpa: Mediapipe assisted (add more points with clicks)
        4. dliba: Dlib assisted (add more points with clicks)
        5. click: Add all points with clicks
        6. txt: Read .txt file for tie pionts of each image
    """

    # Load the images
    img1 = cv2.imread(src_image_path)
    img2 = cv2.imread(dst_image_path)
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
 
    # points1 = find_landmarks(img1)
    # points2 = find_landmarks(img2)
    if tie_points_acquisition == "mp":
            print("Using MediaPipe for landmark points.")
            points1 = find_landmarks_mp(img1)
            points2 = find_landmarks_mp(img2)
           

    elif tie_points_acquisition == "dlib":
            print("Using Dlib for landmark points.")
            points1 = find_landmarks_dlib(img1)
            points2 = find_landmarks_dlib(img2)

    elif tie_points_acquisition == "mpa":
            print("Using Mediapipe + manual clicks for additional points.")
            points1, points2 = mpa(src_image_path, dst_image_path)
            
    elif tie_points_acquisition == "dliba":
            print("Using Dlib + manual clicks for additional points.")
            points1, points2 = dliba(src_image_path, dst_image_path)

    elif tie_points_acquisition == "click":
            print("Manually adding all points with clicks.")
            points1, points2 = clickity(src_image_path, dst_image_path)

    elif tie_points_acquisition == "txt":
            print("Reading tie points from tie.txt")
            points1, points2 = read_ties_from_file('tie.txt')
            
    else:
            raise ValueError("Unsupported tie_points_acquisition option")
    
    
    # Generate the morphing GIF.
    morph_images(img1, img2, points1, points2, num_frames=60, gif_name='morph.gif')

In [ ]:
if __name__ == '__main__':
    src_image_path = "image1.png"
    dst_image_path = "image2.png"
    
    tie_point_acquisition = input("Choose method of tie point acquisition: ")
    
    main(src_image_path, dst_image_path, tie_point_acquisition)